In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Important Dependencies

In [9]:
!pip install -r drive/MyDrive/templates/requirements.txt
!python -m spacy download en_core_web_trf
!sudo apt install tesseract-ocr
!apt-get install -y poppler-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.4) to /tmp/pip-install-vsgnhqkd/detectron2_ecf6846795a24749b8ee67b325f1c5d4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-vsgnhqkd/detectron2_ecf6846795a24749b8ee67b325f1c5d4
  Running command git checkout -q 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Preparing metadata (setup.py) ... done
2023-06-13 10:48:42.020250: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 936.4 kB/s eta 0:00:00
✔ Download and installation successful


**Server Authentcation**

In [10]:
!ngrok config add-authtoken 2QoG11IhsoSCt27K0ZShsZoeVqK_2WmGrQhbEDCfQSJUGq9qi

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Imports

In [11]:
import pickle
import os
import numpy as np
from flask import Flask, render_template, request, flash, redirect, url_for ,jsonify
from flask_ngrok import run_with_ngrok
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin, login_user, LoginManager, login_required, logout_user, current_user
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import InputRequired, Length, ValidationError
from flask_bcrypt import Bcrypt
from drive.MyDrive.templates.Classes import summarization
from drive.MyDrive.templates.Classes import citations

# Models Load


In [12]:
x101configpath= '/content/drive/MyDrive/templates/models/X101/X101.yaml'
x101modelpath = '/content/drive/MyDrive/templates/models/X101/model.pth'

T5generalBase ='/content/drive/MyDrive/templates/models/T5Model'
t5midecalpath= '/content/drive/MyDrive/templates/models/SimpleT5_V6'



#----Citation Section-----------
trfmodel , x101model = citations.Models_Load(x101configpath,x101modelpath)
#----Summarization Section------
berttokenizer,bertmodel = summarization.Bert_Load() #extractive model
t5General_tokenizer,t5General_model = summarization.T5_Load(T5generalBase) #general model
t5Medicaltokenizer,t5Medicalmodel = summarization.T5_Load(t5midecalpath)   #medical model

# Server

In [17]:
db = SQLAlchemy()
app = Flask(__name__, template_folder='/content/drive/MyDrive/templates',static_folder='/content/drive/MyDrive/templates/static')
bcrypt = Bcrypt(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////content/drive/MyDrive/templates/DataBase/database.db'
db.init_app(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(20), nullable=False, unique=True)
    password = db.Column(db.String(80), nullable=False)

class RegisterForm(FlaskForm):
    username = StringField(validators=[
                           InputRequired(), Length(min=4, max=20)], render_kw={"placeholder": "Username"})

    password = PasswordField(validators=[
                             InputRequired(), Length(min=8, max=20)], render_kw={"placeholder": "Password"})

    submit = SubmitField('Register')

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(
            username=username.data).first()
        if existing_user_username:
            raise ValidationError(
                'That username already exists. Please choose a different one.')

class LoginForm(FlaskForm):
    username = StringField(validators=[
                           InputRequired(), Length(min=4, max=20)], render_kw={"placeholder": "Username"})

    password = PasswordField(validators=[
                             InputRequired(), Length(min=8, max=20)], render_kw={"placeholder": "Password"})

    submit = SubmitField('Login')


In [ ]:
#@title Default title text
secret_key = os.urandom(24)
app.secret_key = secret_key
run_with_ngrok(app)

citation_dirc = os.path.join(app.static_folder, 'pdf_files_citation')
sumarization_dirc = os.path.join(app.static_folder, 'pdf_files_summary')

##---------------authentication routing ------------------
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        username = request.form['username']
        password = request.form['password']
        user = User.query.filter_by(username=username).first()
        if user and bcrypt.check_password_hash(user.password, password):
            login_user(user)
            return redirect(url_for('home'))
        else:
            # Invalid credentials
            flash('Invalid username or password.', 'error')
    return render_template('login.html', form=form)

@app.route('/logout', methods=['GET', 'POST'])
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))


@app.route('/register', methods=['GET', 'POST'])
def register():
    form = RegisterForm()

    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))

    return render_template('register.html', form=form)


##----------------navbar routing ---------------------
@app.route('/', methods=['GET'])
def home():
    return render_template('home.html')

@app.route('/citation', methods=['GET'])
@login_required
def citation():
    return render_template('citation.html')

@app.route('/Signin', methods=['GET'])
def signin():
    return render_template('signin.html')

@app.route('/Signup', methods=['GET'])
def signiup():
    return render_template('signup.html')

@app.route('/summarization_page', methods=['GET'])
@login_required
def summarization_page():
    return render_template('summarization.html')
#-----------------------end nav bar ----------------------------------


#-------------------------citationsection end--------------------------------
@app.route('/upload', methods=['POST'])
def upload():
    citations.delete_pdfs_from_directory(citation_dirc)
    file = request.files['file']
    if file:
        filename = file.filename
        file.save(os.path.join(app.static_folder, 'pdf_files_citation', filename))
        flash('File uploaded successfully.', 'success')
    else:
        flash('No file uploaded.', 'error')
    return redirect(url_for('citation'))
@app.route('/predict', methods=['POST'])
def predection():
    citations.delete_img_from_directory("/content/pdf_img")
    author = request.form.get('Author')
    Date = request.form.get('Date')
    Title = request.form.get('Title')
    Publisher = request.form.get('Publisher')
    selected_value = request.form.get('citationStyleSelect')
    finalcitation = None

    if author != '' and Date != '' and Title != '' and Publisher != '' :
        if selected_value == '1':
            finalcitation = citations.make_citation_APA_format(author, Publisher, Date, Title)
        elif selected_value == '2':
            finalcitation = citations.make_citation_MLA_format(author, Publisher, Date, Title)
        elif selected_value == '3':
            finalcitation = citations.make_citation_Chicago_format(author, Publisher, Date, Title)
        else:
            flash('Please choose a style for citation.', 'unsuccess')

    elif selected_value in ['1', '2', '3']:
        img_path = citations.preprocess_pdf(citation_dirc)
        value = np.int32(selected_value)
        finalcitation = citations.predict_citation(img_path, x101model, trfmodel, value)

    else:
        flash('Please fill in all the required fields.', 'unsuccess')
    return render_template('citation.html', finalcitation=finalcitation)
#-------------------------citationsection end--------------------------------


#--------------------sumrization(medical)section------------------------------------
@app.route('/uploadsumary', methods=['POST'])
def uploadsumary():
    citations.delete_pdfs_from_directory(sumarization_dirc)
    files = request.files.getlist('file')
    for file in files:
        filename = file.filename
        file.save(os.path.join(app.static_folder, 'pdf_files_summary', filename))
    citations.delete_img_from_directory('/content/pdf_img')
    flash('File uploaded successfully.', 'success')
    return redirect(url_for('summarization_page'))

@app.route('/predictSummary', methods=['POST'])
def summary():
  selected_value = request.form.get('summaryStyleSelect')
  print("selected_value",selected_value)
  textarea_value = request.form.get('summaryErea')
  print("textarea_value",textarea_value)
  if textarea_value :
    if selected_value == '1':
      final_summary =summarization.Do_Summarization(textarea_value , berttokenizer , bertmodel ,t5Medicaltokenizer ,t5Medicalmodel)
      return render_template('summarization.html', final_summary=final_summary)
    elif selected_value =='2':
      final_summary =summarization.Do_Summarization(textarea_value , berttokenizer , bertmodel , t5General_tokenizer ,t5General_model)
      return render_template('summarization.html', final_summary=final_summary)
    else:
      flash('Please choose a topic for summarization.', 'unsuccess')
      return render_template('summarization.html')
  else:
    if selected_value == '1':
      img_path = citations.preprocess_pdf(sumarization_dirc)
      abstract_str=citations.Extract_Abstract(img_path,x101model)

      final_summary =summarization.Do_Summarization(abstract_str , berttokenizer , bertmodel , t5Medicaltokenizer,t5Medicalmodel)
      citations.delete_img_from_directory(img_path)
      return render_template('summarization.html', final_summary=final_summary)
    elif selected_value =='2':
      img_path = citations.preprocess_pdf(sumarization_dirc)
      abstract_str=citations.Extract_Abstract(img_path,x101model)

      final_summary =summarization.Do_Summarization(abstract_str , berttokenizer , bertmodel , t5General_tokenizer ,t5General_model)
      citations.delete_img_from_directory(img_path)
      return render_template('summarization.html', final_summary=final_summary)
    else:
      flash('Please choose a topic for summarization.', 'unsuccess')
      return render_template('summarization.html')
#--------------------sumrization(medical)section------------------------------------

#--------------------footersection------------------------------------------

#--------------------footersection------------------------------------------


if __name__ == '__main__':

    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e1e9-35-204-4-52.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:32] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:32] "GET /static/script.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:32] "GET /static/assets/icons/the_Papertown-removebg-preview.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:32] "GET /static/assets/images/wave_4.svg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:33] "GET /static/assets/images/bookz.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:35] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:36] "GET /static/signs.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:36] "GET /static/assets/images/sign.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Jun/2023 11:45:43] 

selected_value 1
textarea_value 


100%|██████████| 5/5 [00:00<00:00, 26.42it/s]
